In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

In [ ]:
# Set up paths

path_to_data = '/data/projects/Comet/cwp140/'      # project data -- read only
path_to_out  = '../out/'       # output files (numerical results, intermediate datafiles) -- read & write
path_to_figs = '../figs/'      # figures

In [ ]:
# fname = path_to_data + 'downloads/SEAK_flood_catalog.csv'
fname = '/data/projects/Comet/cwp140/downloads/ar_impact_info.csv'
df = pd.read_csv(fname)
# some cleaning
# df = df.drop(['Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9','Unnamed: 10', 'Unnamed: 11','Unnamed: 12','Unnamed: 13','Unnamed: 14'], axis=1)
df

In [ ]:
# set start and end dates to match WRF data
start_date = '1980-01-01 0:00'
end_date = '2022-12-31 23:00'

In [ ]:
def get_impact_dates_from_aaron_df(df, freq):
    
    if freq == '1D':
        impactdates = df['Impact dates'].values
    
    elif freq == '1H':
        dt_lst = []
        for index, row in df.iterrows():
            current_date = row['Impact dates']
            dt = datetime.strptime(current_date, "%m/%d/%Y")
            dt = pd.date_range(dt, dt + timedelta(hours=23), freq=freq)
            dt_lst.append(dt)
        impactdates = dt_lst[0].union_many(dt_lst[1:])
    
    
    # put into dataframe
    d = {'impactdates': impactdates}
    df = pd.DataFrame(data=d)

    # keep only unique dates
    df.index = pd.to_datetime(df['impactdates'])
    df_new = df['impactdates'].unique()
    # make a new df with only unique dates
    d = {'dates': df_new}
    df = pd.DataFrame(data=d)
    df.index = pd.to_datetime(df.dates)
    df = df.drop(['dates'], axis=1)
    df = df.reset_index()
    df['impact'] = 1


    # date array with all days
    dates_allDays = pd.date_range(start=start_date, end=end_date, freq=freq)
    arr_allDays = np.zeros(len(dates_allDays), dtype=int)

    # Loop over ar days and match to ar_full 
    for i, date in enumerate(df['dates'].values):
        idx = np.where(dates_allDays == date)
        arr_allDays[idx] = 1

    # Create dataframe
    data = {'IMPACT':arr_allDays}
    df_all = pd.DataFrame(data, index=dates_allDays)
    
    return df_all

In [ ]:
impactdates_hourly = get_impact_dates_from_aaron_df(df, '1H')
impactdates_daily = get_impact_dates_from_aaron_df(df, '1D')

In [ ]:
## save to csv file
impactdates_hourly.to_csv(path_to_out + 'SEAK_impactdates_hourly.csv')
impactdates_daily.to_csv(path_to_out + 'SEAK_impactdates_daily.csv')